# Проведем эксперимент с теми же текстами, но очищенными от стоп-слов

In [10]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [11]:
nltk.download("stopwords")
stops = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
bible = pd.read_csv('/content/prep_asb_SW.csv')


In [3]:
bible.head()

,Unnamed: 0,Text,cleaned_text,prep_text,len
0,0,In the beginning God created the heavens and t...,in the beginning god created the heavens and t...,in the beginning god created the heavens and t...,55
1,1,And the earth was waste and void; and darkness...,and the earth was waste and void and darkness...,and the earth was waste and void and darkness...,134
2,2,"And God said, Let there be light: and there wa...",and god said let there be light and there wa...,and god said let there be light and there wa...,54
3,3,"And God saw the light, that it was good: and G...",and god saw the light that it was good and g...,and god saw the light that it was good and g...,85
4,4,"And God called the light Day, and the darkness...",and god called the light day and the darkness...,and god called the light day and the darkness...,63


In [4]:
bible['prep_text'] = bible['prep_text'].astype(str)

In [12]:
import string

def stopwords_remover(words):

  without_stopwords = ""
  for word in words.split(" "):
    if word in stops or word in string.punctuation:
      continue
    else:
      without_stopwords += word + " "
  return without_stopwords

In [13]:
bible['prep_text'] = bible['prep_text'].apply(stopwords_remover)

In [14]:
def count_words(df):
    w = 0
    for idx, row in df.iterrows():
        w+= len(row['prep_text'].split())
    return w

print("Bible", count_words(bible))


Bible 377272


In [15]:
bible

,Unnamed: 0,Text,cleaned_text,prep_text,len
0,0,In the beginning God created the heavens and t...,in the beginning god created the heavens and t...,beginning god created heavens earth,55
1,1,And the earth was waste and void; and darkness...,and the earth was waste and void and darkness...,earth waste void darkness upon face deep spiri...,134
2,2,"And God said, Let there be light: and there wa...",and god said let there be light and there wa...,god said let light light,54
3,3,"And God saw the light, that it was good: and G...",and god saw the light that it was good and g...,god saw light good god divided light darkness,85
4,4,"And God called the light Day, and the darkness...",and god called the light day and the darkness...,god called light day darkness called night,63
...,...,...,...,...,...
35345,35345,and if any man shall take away from the words ...,and if any man shall take away from the words ...,man shall take away words book prophecy god sh...,185
35346,35346,"He who testifieth these things saith, Yea: I c...",he who testifieth these things saith yea i c...,testifieth things saith yea come quickly,58
35347,35347,"Amen: come, Lord Jesus.",amen come lord jesus,amen come lord jesus,23
35348,35348,The grace of the Lord Jesus be with the saints.,the grace of the lord jesus be with the saints,grace lord jesus saints,47


###BIOLOGY_ACADEMIC

In [16]:
bio = pd.read_csv('/content/prep_BIO_SW.csv')
bio.head()

,Unnamed: 0,text,cleaned_text,removed_terms_text,len
0,0,bs_bs_banner\n\nINVITED REVIEW SERIES:\nSTEM C...,bs bs banner invited review series stem cells ...,bs bs banner invited review series stem cells ...,140
1,1,IVAN BERTONCELLO AND JONATHAN L. MCQUALTER\n\n...,ivan bertoncello and jonathan mcqualter lung h...,and jonathan health research centre department...,494
2,2,"In this\nreview, we describe the attributes of...",in this review we describe the attributes of a...,in this review we describe the of adult stem a...,191
3,3,We describe the power and\nlimitations of expe...,we describe the power and limitations of exper...,we describe the power and of strategies and us...,214
4,4,The review summarizes\nrecent progress and obs...,the review summarizes recent progress and obst...,the review summarizes recent progress and in d...,161


In [17]:
bio.rename({"removed_terms_text":"prep_text"}, axis=1, inplace=True) # переименуем колонку

In [18]:
bio['prep_text'] = bio['prep_text'].astype(str)

In [19]:
bio['prep_text'] = bio['prep_text'].apply(stopwords_remover)

In [20]:
print("bio", count_words(bio))

bio 593023


In [21]:
bible['target'] = 0
bio['target'] = 1

In [22]:
n = len(bible)
bio = bio.iloc[:n]
bible = bible.iloc[:n]

In [27]:
corpus = pd.concat([bible[['prep_text', 'target']], bio[['prep_text', 'target']]], axis=0).dropna()
corpus

,prep_text,target
0,beginning god created heavens earth,0
1,earth waste void darkness upon face deep spiri...,0
2,god said let light light,0
3,god saw light good god divided light darkness,0
4,god called light day darkness called night,0
...,...,...
35345,primarily contend discriminate isotopic replaced,1
35346,although preserve shape known change consequen...,1
35347,meanwhile shape argue replaced different origi...,1
35348,therefore interaction binding inside receptor ...,1


In [28]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device available for running: ")
print(device)

Device available for running: 
cuda


In [29]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True)
model_tf = DistilBertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from sklearn.model_selection import train_test_split

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(corpus['prep_text'],
                                                                corpus['target'],
                                                                test_size=0.2,
                                                                shuffle=True)

In [31]:
from torch.utils.data import TensorDataset, DataLoader

train_encodings = tokenizer(list(X_train_tf.values), padding=True)
test_encodings = tokenizer(list(X_test_tf.values), padding=True)

In [32]:
train_labels = torch.tensor(y_train_tf.values)
test_labels = torch.tensor(y_test_tf.values)


train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)

In [33]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [34]:
from tqdm import tqdm
from torch import nn
from torch.optim import Adam, AdamW

In [35]:
! pip install accelerate
from accelerate import Accelerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [36]:
accelerator = Accelerator(mixed_precision="fp16")
optimizer_tf = AdamW(model_tf.parameters(), lr=2e-5)

model, optimizer, train_loader = accelerator.prepare(model_tf, optimizer_tf, train_loader)

model.train()

epochs = 5
criterion = nn.CrossEntropyLoss()
for epoch in tqdm(range(epochs)):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        output = model(texts, attention_mask=atts)

        loss = criterion(output.logits, labels)
        accelerator.backward(loss)
        epoch_loss += loss
        optimizer.step()
    print(f"Train loss: {epoch_loss}")

 20%|██        | 1/5 [07:52<31:31, 472.80s/it]

Train loss: 217.99044799804688


 40%|████      | 2/5 [15:27<23:06, 462.20s/it]

Train loss: 120.15656280517578


 60%|██████    | 3/5 [23:03<15:18, 459.33s/it]

Train loss: 98.64920806884766


 80%|████████  | 4/5 [30:41<07:38, 458.76s/it]

Train loss: 88.6309814453125


100%|██████████| 5/5 [38:18<00:00, 459.71s/it]

Train loss: 83.59625244140625


In [37]:
predictions = []
probabilities = []
original_labels = []

with torch.no_grad():
    model.eval()
    for batch in test_loader:
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        output = model(texts, attention_mask=atts)
        probabilities.extend(output.logits.cpu())
        predictions.extend(output.logits.argmax(axis=1).cpu())
        original_labels.extend(labels.cpu())

In [38]:
from sklearn.metrics import classification_report

print(classification_report(original_labels, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      7151
           1       0.99      1.00      0.99      6989

    accuracy                           0.99     14140
   macro avg       0.99      0.99      0.99     14140
weighted avg       0.99      0.99      0.99     14140



In [75]:
torch.save(model.state_dict(), '/content/sentence_bert_NO_SW.pth')



# Смотрим, как модель определяет библейские цитаты

In [40]:
econ = pd.read_csv("/content/prep_bio_Economist_SW.csv")
econ

,Unnamed: 0,Text,cleaned_text,len
0,0,FOR THE past four billion years or so the only...,for the past four billion years or so the only...,154
1,1,Sometimes the gene would be damaged or scrambl...,sometimes the gene would be damaged or scrambl...,96
2,2,From that raw material arose the glories of na...,from that raw material arose the glories of na...,62
3,3,"But beneath it all, gene begat gene.That is no...",but beneath it all gene begat gene that is no ...,58
4,4,Now genes can be written from scratch and edit...,now genes can be written from scratch and edit...,90
...,...,...,...,...
35847,35972,"He found that, after 15 days, parasite loads i...",he found that after days parasite loads in ani...,269
35848,35973,Dr Kumar plans to try—though he will use genet...,dr kumar plans to try though he will use genet...,137
35849,35974,"But many, probably most, doctors are suspiciou...",but many probably most doctors are suspicious ...,165
35850,35975,"Dr Kumar’s findings are, nevertheless, interes...",dr kumar s findings are nevertheless interesting,49


In [62]:
econ['cleaned_text'] = econ['cleaned_text'].astype(str)

In [65]:
econ = econ[econ['cleaned_text'].apply(lambda x: isinstance(x, str))]

In [67]:
econ['cleaned_text'] = econ['cleaned_text'].apply(stopwords_remover)

In [68]:
infer_encodings = tokenizer(list(econ["cleaned_text"].values), padding=True)
infer_dataset = TensorDataset(torch.tensor(infer_encodings['input_ids']),
                              torch.tensor(infer_encodings['attention_mask']))

infer_loader = DataLoader(infer_dataset, batch_size=16, shuffle=False)

predictions = []
probabilities = []

with torch.no_grad():
    model.eval()
    for batch in infer_loader:
        texts, atts = batch[0].to(device), batch[1].to(device)
        output = model(texts, attention_mask=atts)
        probabilities.extend(output.logits.cpu())
        predictions.extend(output.logits.argmax(axis=1).cpu())

In [69]:
predictions

[tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 ten

In [70]:
np.array(predictions) == 0

array([False, False,  True, ...,  True, False, False])

In [71]:
predictions = np.array(predictions)

In [72]:
econ[predictions == 0].to_csv('/content/sentence_NO_SW_result.csv')

In [73]:
import numpy as np
import pandas as pd

In [74]:
Res = pd.read_csv('/content/sentence_NO_SW_result.csv')
Res

,Unnamed: 0.1,Unnamed: 0,Text,cleaned_text,len
0,2,2,From that raw material arose the glories of na...,raw material arose glories natural selection,62
1,6,6,It permits the manufacture of all manner of th...,permits manufacture manner things used hard ev...,186
2,12,12,"But look back through history, and humanity’s ...",look back history humanity relations living wo...,306
3,13,13,"All brought prosperity and progress, but with ...",brought prosperity progress damaging side effe...,67
4,17,17,"But much wilderness has been lost, and carbon ...",much wilderness lost carbon atoms last saw atm...,204
...,...,...,...,...,...
8794,35817,35942,Their manufacturers hope that charging a “gree...,manufacturers hope charging green premium make...,86
8795,35824,35949,ONE of the most valuable weapons in the war on...,one valuable weapons war malaria artemisinin d...,119
8796,35828,35953,"Moreover, yields are not huge—a mere half a mi...",moreover yields huge mere half milligram per g...,87
8797,35844,35969,"Moreover, they produced 0.8 milligrams of arte...",moreover produced milligrams artemisinin gram ...,201


In [76]:
Res = Res.drop('Unnamed: 0.1', axis=1)

In [77]:
Res = Res.drop('Unnamed: 0', axis=1)

In [78]:
Res

,Text,cleaned_text,len
0,From that raw material arose the glories of na...,raw material arose glories natural selection,62
1,It permits the manufacture of all manner of th...,permits manufacture manner things used hard ev...,186
2,"But look back through history, and humanity’s ...",look back history humanity relations living wo...,306
3,"All brought prosperity and progress, but with ...",brought prosperity progress damaging side effe...,67
4,"But much wilderness has been lost, and carbon ...",much wilderness lost carbon atoms last saw atm...,204
...,...,...,...
8794,Their manufacturers hope that charging a “gree...,manufacturers hope charging green premium make...,86
8795,ONE of the most valuable weapons in the war on...,one valuable weapons war malaria artemisinin d...,119
8796,"Moreover, yields are not huge—a mere half a mi...",moreover yields huge mere half milligram per g...,87
8797,"Moreover, they produced 0.8 milligrams of arte...",moreover produced milligrams artemisinin gram ...,201


In [79]:
Res['Text'] = Res['Text'].astype(str)

In [80]:
Res.loc[0].Text

'From that raw material arose the glories of natural selection.'

In [81]:
Res.loc[5].Text

'Horses, cattle and cotton were introduced to the Americas; maize, potatoes, chilli and tobacco to Europe, Africa and Asia.'

In [82]:
Res.loc[20].Text

'As Isaac Newton put it, “If I have seen further, it is by standing on the shoulders of giants.” If the brains of those giants happen to be made of silicon chips, so be it.'

In [83]:
Res.loc[40].Text

'But they may not always need living things to produce them.'

In [85]:
Res.loc[100].Text

'But it is right to wonder what might have happened if they had—and what should be done if, or when, someone decides to try.'